In [2]:
import torch
import tensorrt as trt

print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())
print("TensorRT:", trt.__version__)

Torch: 2.9.1+cu128 CUDA: True
TensorRT: 10.14.1.48.post1


In [3]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import tensorrt as trt
import numpy as np

sys.path.insert(0, str(Path.cwd().parents[1]))

from models.resnet32_model import ResNet, ResNetQAT

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [15]:
import torch
import torch.quantization as tq

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.quantized.engine = "fbgemm"  # important if you used fbgemm in training

# 1) Build model
model = ResNetQAT(num_classes=10)

# 2) Prepare QAT graph (must be in train mode for prepare_qat)
model.train()
model.qconfig = tq.get_default_qat_qconfig("fbgemm")
tq.prepare_qat(model, inplace=True)

# 3) Load checkpoint BUT drop activation_post_process_* keys (they don't match your naming)
state = torch.load("../../pth/resnet_qat_preconvert.pth", map_location="cpu")
state = {k: v for k, v in state.items() if not k.startswith("activation_post_process_")}

missing, unexpected = model.load_state_dict(state, strict=False)
print("missing:", len(missing), "unexpected:", len(unexpected))

# 4) Calibrate observers (collect min/max) WITHOUT fake quant affecting activations yet
model.to(device)
model.train()
model.apply(tq.enable_observer)
model.apply(tq.disable_fake_quant)

with torch.no_grad():
    for i, (x, _) in enumerate(test_loader):
        x = x.to(device)
        _ = model(x)
        if i == 50:   # ~50 batches is plenty
            break

# 5) Now run inference with fake quant enabled and observers frozen
model.eval()
model.apply(tq.disable_observer)
model.apply(tq.enable_fake_quant)

missing: 676 unexpected: 165


/tmp/SLURM_5287623/ipykernel_2244149/1867164493.py:13: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  tq.prepare_qat(model, inplace=True)
/home1/ihsiao/.conda/envs/ee599/lib/python3.12/site-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated 

ResNetQAT(
  (conv1): Conv2d(
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1], device='cuda:0'), observer_enabled=tensor([0], device='cuda:0'), scale=tensor([0.0070, 0.0007, 0.0011, 0.0032, 0.0062, 0.0042, 0.0023, 0.0022, 0.0016,
              0.0021, 0.0015, 0.0012, 0.0021, 0.0019, 0.0020, 0.0052],
             device='cuda:0'), zero_point=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0',
             dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(
        min_val=tensor([-0.8860, -0.1091, -0.2391, -0.4209, -0.7691, -0.6686, -0.1984, -0.4251,
                -0.3105, -0.2907, -0.3131, -0.2347, -0.2437, -0.3173, -0.2896, -0.6297],
               device='cuda:0'), max_val=tensor([0.3645, 0.1739, 0.243

In [18]:
import torch
import torch.ao.quantization as tq

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Put model on GPU for calibration/inference of QAT (fake-quant runs fine on GPU)
model.to(device)
model.eval()

# 2) IMPORTANT: freeze observers + enable fake quant BEFORE export
model.apply(tq.disable_observer)
model.apply(tq.enable_fake_quant)

dummy_map = {
    1:   torch.randn(1,   3, 32, 32, device=device),
    64:  torch.randn(64,  3, 32, 32, device=device),
    128: torch.randn(128, 3, 32, 32, device=device),
}

for bs, dummy in dummy_map.items():
    out_path = f"resnet_qat_qdq_b{bs}_op18.onnx"
    torch.onnx.export(
        model, dummy, out_path,
        opset_version=18,
        do_constant_folding=False,
        input_names=["input"],
        output_names=["logits"],
        dynamic_axes=None,
    )
    print("Exported", out_path)

[torch.onnx] Obtain model graph for `ResNetQAT([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNetQAT([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ❌


ConversionError: Failed to convert the exported program to an ONNX model. [96mThis is step 3/3[0m of exporting the model to ONNX. Next steps:
- If there is a missing ONNX function, implement it and register it to the registry.
- If there is an internal error during ONNX conversion, debug the error and summit a PR to PyTorch.
- Create an error report with `torch.onnx.export(..., report=True)`, and save the ExportedProgram as a pt2 file. Create an issue in the PyTorch GitHub repository against the [96m*onnx*[0m component. Attach the error report and the pt2 model.

## Exception summary

<class 'torch.onnx._internal.exporter._errors.DispatchError'>: No ONNX function found for <OpOverload(op='aten._fused_moving_avg_obs_fq_helper_functional', overload='default')>. Failure message: No decompositions registered for the real-valued input
⬆️
<class 'torch.onnx._internal.exporter._errors.ConversionError'>: Error when translating node %_fused_moving_avg_obs_fq_helper_functional : [num_users=5

In [10]:
!ls -lh resnet_qat_int8_b1_op18.onnx
!ls -lh resnet_qat_int8_b64_op18.onnx
!ls -lh resnet_qat_int8_b128_op18.onnx

-rw-r--r-- 1 ihsiao ihsiao 78K Dec 14 09:47 resnet_qat_int8_b1_op18.onnx
-rw-r--r-- 1 ihsiao ihsiao 78K Dec 14 09:47 resnet_qat_int8_b64_op18.onnx
-rw-r--r-- 1 ihsiao ihsiao 78K Dec 14 09:47 resnet_qat_int8_b128_op18.onnx


In [11]:
import onnx
m = onnx.load("resnet_qat_int8_b1_op13.onnx")
onnx.checker.check_model(m)
print([(op.domain, op.version) for op in m.opset_import])

[('', 18)]


In [12]:
import tensorrt as trt
print(trt.__version__)

10.14.1.48.post1


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [14]:
from torch.utils.data import DataLoader

test_loader_b1   = DataLoader(test_dataset, batch_size=1,   shuffle=False, num_workers=2, pin_memory=True, drop_last=True)
test_loader_b64  = DataLoader(test_dataset, batch_size=64,  shuffle=False, num_workers=2, pin_memory=True, drop_last=True)
test_loader_b128 = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True, drop_last=True)

In [ ]:
import torch
import tensorrt as trt

@torch.no_grad()
def trt_accuracy_static(engine_path, test_loader, num_batches=None):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

    with open(engine_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        engine = runtime.deserialize_cuda_engine(f.read())
    context = engine.create_execution_context()

    names = [engine.get_tensor_name(i) for i in range(engine.num_io_tensors)]
    inp = [n for n in names if engine.get_tensor_mode(n) == trt.TensorIOMode.INPUT][0]
    out = [n for n in names if engine.get_tensor_mode(n) == trt.TensorIOMode.OUTPUT][0]

    # engine fixed shapes
    in_shape = tuple(engine.get_tensor_shape(inp))
    out_shape = tuple(engine.get_tensor_shape(out))
    fixed_bsz = in_shape[0]  # should be 1 or 64 or 128

    # output dtype
    trt_dtype = engine.get_tensor_dtype(out)
    torch_dtype = {
        trt.DataType.FLOAT: torch.float32,
        trt.DataType.HALF:  torch.float16,
        trt.DataType.INT8:  torch.int8,
        trt.DataType.INT32: torch.int32,
    }[trt_dtype]

    stream = torch.cuda.current_stream()
    correct = 0
    total = 0

    for bi, (x_cpu, y_cpu) in enumerate(test_loader):
        if num_batches is not None and bi >= num_batches:
            break

        x = x_cpu.to("cuda", non_blocking=True)
        y = y_cpu.to("cuda", non_blocking=True)

        if x.shape[0] != fixed_bsz:
            raise RuntimeError(f"Batch mismatch: loader={x.shape[0]} but engine expects {fixed_bsz}")

        yhat = torch.empty(out_shape, device="cuda", dtype=torch_dtype)

        context.set_tensor_address(inp, int(x.data_ptr()))
        context.set_tensor_address(out, int(yhat.data_ptr()))

        ok = context.execute_async_v3(stream_handle=stream.cuda_stream)
        if not ok:
            raise RuntimeError("TRT execute failed")

        pred = yhat.float().argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.shape[0]

    torch.cuda.synchronize()
    return 100.0 * correct / total

In [ ]:
import tensorrt as trt
import torch
import onnx

# ---- QUIET logger (prevents per-batch spam) ----
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# ---- YOUR 3 STATIC ONNX FILES (fixed batch) ----
onnx_map = {
    1:   "resnet_qat_fp32_b1_op13.onnx",
    64:  "resnet_qat_fp32_b64_op13.onnx",
    128: "resnet_qat_fp32_b128_op13.onnx",
}

# ---- CALIBRATION LOADERS MUST MATCH BATCH SIZE ----
calib_loader_map = {
    1:   test_loader_b1,
    64:  test_loader_b64,
    128: test_loader_b128,
}

CALIB_BATCHES = 200          # 200 is fine; 500 if you want extra
WORKSPACE     = 1 << 28      # 256MB (raise to 1<<29 if build fails)

# ---------------- Calibrator ----------------
class EntropyCalibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, calib_loader, max_batches=200, cache_file="calib.cache"):
        super().__init__()
        self.cache_file = cache_file
        self.data_iter = iter(calib_loader)
        self.max_batches = max_batches
        self.batch_count = 0

        x0, _ = next(iter(calib_loader))
        self.batch_size = x0.shape[0]
        self.device_input = torch.empty_like(x0, device="cuda")

    def get_batch_size(self):
        return self.batch_size

    def get_batch(self, names):
        if self.batch_count >= self.max_batches:
            return None
        try:
            x, _ = next(self.data_iter)
        except StopIteration:
            return None

        self.batch_count += 1
        x = x.to("cuda", non_blocking=True)
        self.device_input.resize_(x.shape).copy_(x)
        return [int(self.device_input.data_ptr())]

    def read_calibration_cache(self):
        try:
            with open(self.cache_file, "rb") as f:
                return f.read()
        except FileNotFoundError:
            return None

    def write_calibration_cache(self, cache):
        with open(self.cache_file, "wb") as f:
            f.write(cache)

# ---------------- Builder ----------------
def build_int8_engine_static(onnx_path, engine_path, calib_loader, max_calib_batches=200):
    # sanity check: if ONNX already has Q/DQ, you generally should NOT calibrate
    m = onnx.load(onnx_path)
    ops = set(n.op_type for n in m.graph.node)
    has_qdq = ("QuantizeLinear" in ops) or ("DequantizeLinear" in ops)
    if has_qdq:
        print(f"[WARN] {onnx_path} contains Quantize/Dequantize ops. "
              f"PTQ calibration may be wrong for this file.")

    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:

        with open(onnx_path, "rb") as f:
            if not parser.parse(f.read()):
                for i in range(parser.num_errors):
                    print(parser.get_error(i))
                raise RuntimeError(f"ONNX parse failed for {onnx_path}")

        config = builder.create_builder_config()
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, WORKSPACE)

        # INT8 PTQ
        config.set_flag(trt.BuilderFlag.INT8)
        cache_file = engine_path.replace(".engine", ".cache")
        config.int8_calibrator = EntropyCalibrator(
            calib_loader,
            max_batches=max_calib_batches,
            cache_file=cache_file
        )

        # static ONNX => NO optimization profile needed
        serialized = builder.build_serialized_network(network, config)
        if serialized is None:
            raise RuntimeError(f"INT8 engine build failed for {onnx_path}")

        with open(engine_path, "wb") as f:
            f.write(serialized)

    print(f"Saved: {engine_path} | cache: {cache_file} | calib_batches={max_calib_batches}")

# ---------------- Build all 3 ----------------
for bs, onnx_path in onnx_map.items():
    build_int8_engine_static(
        onnx_path=onnx_path,
        engine_path=f"resnet_qat_int8_b{bs}.engine",
        calib_loader=calib_loader_map[bs],
        max_calib_batches=CALIB_BATCHES
    )

In [ ]:
!ls -lh resnet_qat_int8_b1.engine
!ls -lh resnet_qat_int8_b64.engine
!ls -lh resnet_qat_int8_b128.engine

In [ ]:
acc1   = trt_accuracy_static("resnet_qat_int8_b1.engine",   test_loader_b1)
acc64  = trt_accuracy_static("resnet_qat_int8_b64.engine",  test_loader_b64)
acc128 = trt_accuracy_static("resnet_qat_int8_b128.engine", test_loader_b128)

print(f"INT8 TRT Acc b1:   {acc1:.2f}%")
print(f"INT8 TRT Acc b64:  {acc64:.2f}%")
print(f"INT8 TRT Acc b128: {acc128:.2f}%")

In [ ]:
@torch.no_grad()
def torch_acc(model_qat_prepared, loader, device="cuda"):
    model_qat_prepared.eval().to(device)
    correct = total = 0
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        correct += (pred==y).sum().item()
        total += y.size(0)
    return 100*correct/total

print("Torch acc:", torch_acc(model_qat_prepared, test_loader))

In [1]:
print(test_loader.dataset.transform)

NameError: name 'test_loader' is not defined

In [ ]:
import onnx
m = onnx.load("resnet_qat_fp32_b1_op13.onnx")  # <-- change to your qat onnx filename
ops = sorted(set(n.op_type for n in m.graph.node))
print("QuantizeLinear:", "QuantizeLinear" in ops)
print("DequantizeLinear:", "DequantizeLinear" in ops)
print("Num nodes:", len(m.graph.node))

In [ ]:
import onnx
m = onnx.load("resnet_qat_fp32_b1_op13.onnx")
print("Inputs:")
for i in m.graph.input:
    print(i.name)
print("Outputs:")
for o in m.graph.output:
    print(o.name)

In [ ]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
with open("resnet_qat_int8_b1.engine","rb") as f, trt.Runtime(TRT_LOGGER) as rt:
    engine = rt.deserialize_cuda_engine(f.read())

print("Engine name:", engine.name)
print("Has refittable:", engine.refittable)
print("Num layers:", engine.num_layers)
print("Has dynamic shapes:", engine.num_optimization_profiles > 0)

In [ ]:
import onnx
m = onnx.load("resnet_qat_fp32_b1_op13.onnx")   # <-- whichever file you're using
ops = sorted(set(n.op_type for n in m.graph.node))
print("Has QuantizeLinear?", "QuantizeLinear" in ops)
print("Has DequantizeLinear?", "DequantizeLinear" in ops)

In [ ]:
print("example keys:", list(state.keys())[:20])
missing, unexpected = model.load_state_dict(state, strict=False)
print("missing sample:", missing[:20])
print("unexpected sample:", unexpected[:20])

In [ ]:
import torch

state = torch.load("../../pth/resnet_qat_preconvert.pth", map_location="cpu")
print("num keys:", len(state))
print("first 30 keys:")
for k in list(state.keys())[:30]:
    print(" ", k)

In [ ]:
import torch.quantization as tq
from models.resnet32_model import ResNetQAT

model = ResNetQAT(num_classes=10)
model.train()
model.qconfig = tq.get_default_qat_qconfig("fbgemm")
model_qat_prepared = tq.prepare_qat(model, inplace=False)

prepared_keys = set(model_qat_prepared.state_dict().keys())
ckpt_keys = set(state.keys())

print("prepared keys:", len(prepared_keys))
print("ckpt keys:", len(ckpt_keys))
print("overlap:", len(prepared_keys & ckpt_keys))

print("\nmissing (need in ckpt):")
for k in list(prepared_keys - ckpt_keys)[:30]:
    print(" ", k)

print("\nunexpected (in ckpt but not in model):")
for k in list(ckpt_keys - prepared_keys)[:30]:
    print(" ", k)